In [6]:
import pandas as pd
from src.models.load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton

In [11]:
detail = pd.read_pickle('./data/processed/balance_s.pkl')
advised_pf = pd.read_pickle('./data/processed/advised_portfolios.pkl')

In [10]:
detail

,date,userid,name,asset_class,itemcode,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,original
0,3/21/2021 4:00:00 PM,A001,투자자1,Cash,C000001,현금,700000,1,700000,1.00,700000.00,1.000000,20210321 16:00현금성,Y
1,3/22/2021 4:00:00 PM,A001,투자자1,Cash,C000001,현금,7920,1,7920,1.00,7920.00,0.011016,20210322 16:00현금성,N
2,3/22/2021 4:00:00 PM,A001,투자자1,Equity,A069500,KODEX 200,3,42095,42095,42936.90,128810.70,0.179162,20210322 16:00주식,N
3,3/22/2021 4:00:00 PM,A001,투자자1,Equity,A360200,KINDEX 미국S&P500,4,10865,43460,10321.75,41287.00,0.057426,20210322 16:00주식,N
4,3/22/2021 4:00:00 PM,A001,투자자1,Fixed Income,A196230,KBSTAR 단기통안채,1,104325,104325,101195.25,101195.25,0.140752,20210322 16:00채권,N
5,3/22/2021 4:00:00 PM,A001,투자자1,Fixed Income,A302190,TIGER 중장기국채,3,52365,52365,54459.60,163378.80,0.227242,20210322 16:00채권,N
6,3/22/2021 4:00:00 PM,A001,투자자1,Alternative,A329750,TIGER 미국달러단기채권액티브,16,9645,38580,9645.00,154320.00,0.214642,20210322 16:00대체,N
7,3/22/2021 4:00:00 PM,A001,투자자1,Fixed Income,A114260,KODEX 국고채3년,2,58120,58120,61026.00,122052.00,0.169761,20210322 16:00채권,N


In [12]:
current_date='2021-03-22'
risk_profile = 4

In [14]:
advised_pf.loc[(advised_pf.risk_profile == 4) & (
        advised_pf.date > current_date), 'date'].min()

'2021-03-23'

In [ ]:
    def rebalance(self, rebal_date, price_d, detail, new_port):
        '''
        Rebalance a portfolio.
        
        Parameters:
        rebal_date: str
            rebalancing date
        
        detail: DataFrame
        current balance
        
        price_d: DataFrame
        price data on rebal_date
        
        new_port: DataFrame
        A new portfolio. Your current portfolio in `detail` will be rebalanced toward `new_port`.
        '''
        trading_amt = detail.value.sum()       
        
        wt = new_port[['itemcode', 'weights']].set_index('itemcode').to_dict()['weights']
        pr = new_port[['itemcode', 'price']].set_index('itemcode').squeeze()

        da = DiscreteAllocation(weights=wt, latest_prices=pr, total_portfolio_value=trading_amt)
        
        allocation, remaining_cash = da.greedy_portfolio()
        print("리밸런싱 결과:")
        print("{}: 새 포트폴리오(종목코드:수량)-{}".format(rebal_date,allocation))
        print(" - 매매 후 잔액: {:.2f} KRW".format(remaining_cash))
        
        # 매매한 뒤의 레코드 생성
        df_qty = pd.DataFrame.from_dict(allocation, orient='index', columns=['quantity'])
        next_detail = new_port.merge(df_qty, left_on='itemcode', right_index=True, how='inner')
        next_detail['cost_price'] = next_detail.price.copy()   
        next_detail['cost_value'] = next_detail.cost_price*next_detail.quantity
        next_detail['value'] = next_detail.cost_value.copy()
        
        # 매매하고 남은 돈은 현금으로
        df_cash = {
            'itemcode': 'C000001',
            'quantity': remaining_cash,
            'cost_price': 1,
            'price':1,
            'cost_value': remaining_cash,
            'value': remaining_cash,
            'itemname': '현금',
            'asset_class': 'Cash'
        }
        df_cash = pd.DataFrame.from_dict(df_cash, orient='index').T
        
        next_detail = pd.concat((next_detail[['itemcode', 'quantity', 'cost_price', 'price', 'cost_value', 'value',
        'itemname', 'asset_class']], df_cash), axis=0)
        
        next_detail['wt'] = next_detail.value/next_detail.value.sum()
        next_date = datetime.strptime(rebal_date, '%Y-%m-%d')
        #next_date = str(next_date.month)+'/'+str(next_date.day)+'/'+str(next_date.year)+' 03:30:00 PM'
        next_detail['date'] = next_date
        next_detail.reset_index(drop=True, inplace=True)
        next_detail['group_by'] = ''
        next_detail = pd.merge(next_detail,
                price_d.loc[price_d.date==rebal_date, ['date', 'itemcode']],
                left_on=['date', 'itemcode'],
                right_on=['date', 'itemcode'], how='left')
        next_detail['username'] = self.username
        next_detail['userid'] = self.userid
        next_detail['original'] = 'Rebal'
        next_detail = next_detail.rename(columns={'weights':'wt'})
        next_detail = next_detail[['itemcode', 'quantity', 'cost_price', 'price', 'cost_value', 'value',
            'itemname', 'asset_class', 'date', 'userid', 'username', 'group_by',
            'original', 'wt']]

        return next_detail

In [ ]:
def run_simulation(first_trade=False, new_units=None, prices=None, remaining_cash=None):
    price_db = PriceDB.instance().data

    # 최근 잔고 가져오기
    # 아직 어떤 타입으로 가져오는지 모름
    #detail = self.db.getUserBalance(userid=self.userid)       
    #detail = pd.DataFrame(detail, columns=['date', 'userid', 'name', 'asset_class', 'itemcode', 'itemname',
#                                            'quantity', 'cost_price', 'cost_value', 'price', 'value', 'wt', 'group_by', 'original'])

    # 시뮬레이션 기간은 현재일(current_date) 다음 날부터 추천 포트폴리오가 존재하는 마지막날까지임.
    dates = advised_pf.loc[(advised_pf.risk_profile == risk_profile) & (
        advised_pf.date > current_date), 'date'].min()
    rebal_dates = dates
    print('리밸런싱 일자: ', rebal_dates)

    # return할 때 필요한 첫날의 추천 포트 폴리오와 asset class별 정보 수집
    new_port = advised_pf.loc[(advised_pf.date == rebal_dates) & (
        advised_pf.risk_profile == risk_profile), :]

    first_advised_port = new_port.loc[:, ['weights', 'itemname']].groupby(
        'itemname').sum().reset_index()
    by_assetclass = new_port.loc[:, ['weights', 'asset_class']].groupby(
        'asset_class').sum().sort_values('weights', ascending=False).reset_index()


    # next_detail = copy.deepcopy(detail)
    next_detail = detail
    all_the_nexts = pd.DataFrame(columns=next_detail.columns)
    nexts_list = []
    price_db = price_db.loc[:, ['date', 'price', 'itemcode']]

    # 리밸런싱한다.
    new_port = self.advised_pf.loc[(self.advised_pf.risk_profile==self.risk_profile) & (self.advised_pf.date==dt), ['date', 'itemcode', 'weights', 'itemname', 'price', 'asset_class']]
    next_detail = self.rebalance(rebal_date=dt, price_d=price_d, detail=next_detail, new_port=new_port)

    # all_the_nexts = pd.concat((all_the_nexts, next_detail))
    nexts_list.append(next_detail)

    all_the_nexts = pd.concat(nexts_list, axis=0)

    print('리밸런싱 종료----')
    # 불필요한 컬럼 및 행 삭제
    all_the_nexts = all_the_nexts.loc[all_the_nexts.quantity > 0]
    all_the_nexts = all_the_nexts.reset_index(drop=True)
    all_the_nexts['username'] = self.username

    all_the_generals = all_the_nexts.loc[:,['date', 'wt', 'value', 'asset_class']].sort_values(
                                      ['date'], ascending=True).groupby([
                                          'date', 'asset_class'
                                      ]).sum().reset_index(drop=False)
    print('자산군별 요약 계산 종료----')

    all_the_generals['userid'] = self.userid

    # general 테이블 기록
    self.db.insert_general(all_the_generals)

    # detail 테이블에 기록
    self.db.insert_detail(all_the_nexts)

    print('테이블 업데이트 종료----')

    # del all_the_generals
    # del all_the_nexts

    # investor 테이블 기록
    # self.db.insert_investor(userid=self.userid, name=self.username, profile_code=self.risk_profile)

    return first_advised_port, by_assetclass

In [5]:
advised_pf

,date,risk_profile,itemcode,weights,tracking_code,itemname,price,volume,trading_amt_mln,asset_class
0,2019-01-02,2,A273130,0.240000,ACT_BOND_D4,KODEX 종합채권(AA-이상)액티브,104320.0,26468.0,858.104496,Fixed Income
1,2019-01-02,2,A214980,0.240000,MM_PLUS,KODEX 단기채권PLUS,100050.0,10160.0,508.874866,Fixed Income
2,2019-01-02,2,A196230,0.207716,BOK_D.4,KBSTAR 단기통안채,101423.0,393085.0,16568.430895,Fixed Income
3,2019-01-02,2,A114260,0.100000,KTB_D3,KODEX 국고채3년,55690.0,5449.0,251.127486,Fixed Income
4,2019-01-02,2,A122260,0.092284,BOK_D1,KOSEF 통안채1년,97761.0,42453.0,13238.397304,Fixed Income
...,...,...,...,...,...,...,...,...,...,...
19383,2021-04-30,4,A237370,0.240000,BAL_KO3KTB7,KODEX 배당성장채권혼합,12060.0,5299.0,123.076578,Alternative
19384,2021-04-30,4,A278540,0.240000,MSCI_KR,KODEX MSCI Korea TR,13695.0,471792.0,14353.562789,Equity
19385,2021-04-30,4,A266370,0.136000,IT,KODEX IT,20760.0,21069.0,1087.656470,Equity
19386,2021-04-30,4,A292150,0.084000,TOP10,TIGER TOP10,14305.0,547411.0,14890.029898,Equity


In [47]:
current_date = '2021-1-4'
current_date = datetime.strptime(current_date, '%Y-%m-%d').strftime('%Y-%m-%d')

In [48]:
current_date

'2021-01-04'

In [114]:
df = advised_pf.loc[(advised_pf.date==current_date) & (advised_pf.risk_profile==int('4')), :]

In [115]:
df.tail(30)

,date,risk_profile,itemcode,weights,tracking_code,itemname,price,volume,trading_amt_mln,asset_class
1831,2021-01-04,4,A157490,0.25,SW,TIGER 소프트웨어,14935.0,227434.0,1868.593400,Equity
1832,2021-01-04,4,A139260,0.25,K200_IT,TIGER 200 IT,37650.0,1550986.0,11192.305263,Equity
1833,2021-01-04,4,A266370,0.25,IT,KODEX IT,20805.0,152199.0,539.058393,Equity
1834,2021-01-04,4,A091160,0.20,SEMI_KRX,KODEX 반도체,34355.0,425277.0,2607.676698,Equity
1835,2021-01-04,4,A229200,0.05,KQ150,KODEX 코스닥 150,15400.0,5770313.0,119331.906733,Equity


In [109]:
df.tail(20)

,date,risk_profile,itemcode,weights,tracking_code,itemname,price,volume,trading_amt_mln,asset_class
1820,2021-01-04,2,A122260,0.25,BOK_D1,KOSEF 통안채1년,101015.0,2773.0,53191.510933,Fixed Income
1821,2021-01-04,2,A114260,0.25,KTB_D3,KODEX 국고채3년,58100.0,4571.0,547.479291,Fixed Income
1822,2021-01-04,2,A130730,0.25,MM,KOSEF 단기자금,100835.0,88085.0,59165.272681,Fixed Income
1823,2021-01-04,2,A329650,0.15,BAL_WO3KTB7,KODEX TRF3070,10735.0,33117.0,122.211411,Alternative
1824,2021-01-04,2,A266370,0.10,IT,KODEX IT,20805.0,152199.0,539.058393,Equity


In [106]:
s = '투자자9'

In [107]:
import re

In [108]:
'A' + ('0'+re.findall('\d+', s )[0])[-2:]

'A09'

In [121]:
balance_s.columns

Index(['date', 'userid', 'name', 'asset_class', 'itemcode', 'itemname',
       'quantity', 'cost_price', 'cost_value', 'price', 'value', 'wt',
       'group_by', 'principal'],
      dtype='object')

In [123]:
balance = [
    ('3/22/2021 7:50:25 PM', 'A50', '투자자50', '현금성', 'C000001', '현금', 1000000.0,
     1.0, 1000000.0, 1.0, 1000000.0, 1.0, '2021322:50현금성', 'Y'),
    ('3/22/2021 7:50:25 PM', 'A50', '투자자50', '현금성', 'C000001', '현금', 1000000.0,
     1.0, 1000000.0, 1.0, 1000000.0, 1.0, '2021322:50현금성', 'Y')
]

In [128]:
balance = pd.DataFrame(balance, columns=['date', 'userid', 'name', 'asset_class', 'itemcode', 'itemname',
       'quantity', 'cost_price', 'cost_value', 'price', 'value', 'wt',
       'group_by', 'original'])

In [130]:
balance.drop(['price'], axis=1)

,date,userid,name,asset_class,itemcode,itemname,quantity,cost_price,cost_value,value,wt,group_by,original
0,3/22/2021 7:50:25 PM,A50,투자자50,현금성,C000001,현금,1000000.0,1.0,1000000.0,1000000.0,1.0,2021322:50현금성,Y
1,3/22/2021 7:50:25 PM,A50,투자자50,현금성,C000001,현금,1000000.0,1.0,1000000.0,1000000.0,1.0,2021322:50현금성,Y


In [65]:
import plotly.express as px

In [76]:
pie = px.pie(df, names=df.iloc[:,0], values=df.iloc[:,1])

In [77]:
df

,종목명,비중
0,KODEX IT,0.10
1,KODEX TRF3070,0.15
2,KODEX 국고채3년,0.25
3,KOSEF 단기자금,0.25
4,KOSEF 통안채1년,0.25


In [78]:
pie.show()

In [79]:
df

,종목명,비중
0,KODEX IT,0.10
1,KODEX TRF3070,0.15
2,KODEX 국고채3년,0.25
3,KOSEF 단기자금,0.25
4,KOSEF 통안채1년,0.25


In [39]:
df

,date,risk_profile,itemcode,weights,tracking_code,itemname,price,volume,trading_amt_mln,asset_class
2453,2021-02-26,2,A122260,0.25,BOK_D1,KOSEF 통안채1년,101155.0,829489.0,61672.391133,Fixed Income
2454,2021-02-26,2,A114260,0.25,KTB_D3,KODEX 국고채3년,58090.0,5062.0,821.950469,Fixed Income
2455,2021-02-26,2,A130730,0.25,MM,KOSEF 단기자금,100960.0,965560.0,64369.329522,Fixed Income
2456,2021-02-26,2,A329650,0.15,BAL_WO3KTB7,KODEX TRF3070,10925.0,28428.0,210.266644,Alternative
2457,2021-02-26,2,A266370,0.10,IT,KODEX IT,21480.0,118417.0,1091.216569,Equity


In [44]:
        df.loc[:, ['weights', 'itemname']].groupby(
            'itemname').sum().reset_index().rename(columns={
                'itemcode': '종목명',
                'weights': '비중'
            })

,itemname,비중
0,KODEX IT,0.10
1,KODEX TRF3070,0.15
2,KODEX 국고채3년,0.25
3,KOSEF 단기자금,0.25
4,KOSEF 통안채1년,0.25


In [18]:
print(advised_pf.columns)

Index(['date', 'risk_profile', 'itemcode', 'weights', 'tracking_code',
       'itemname', 'price', 'volume', 'trading_amt_mln', 'asset_class'],
      dtype='object')


In [13]:
current_date='2020-2-1'

In [14]:
advised_pf.date

0      2020-02-03
1      2020-02-03
2      2020-02-03
3      2020-02-03
4      2020-02-03
          ...    
456    2020-02-28
457    2020-02-28
458    2020-02-28
459    2020-02-28
460    2020-02-28
Name: date, Length: 461, dtype: object

In [21]:
for index, row in advised_pf.iterrows():
    print(row.date, row.risk_profile)


2020-08-03 2
2020-08-03 2
2020-08-03 2
2020-08-03 2
2020-08-03 2
2020-08-03 3
2020-08-03 3
2020-08-03 3
2020-08-03 3
2020-08-03 3
2020-08-03 3
2020-08-03 4
2020-08-03 4
2020-08-03 4
2020-08-03 4
2020-08-03 4
2020-08-04 2
2020-08-04 2
2020-08-04 2
2020-08-04 2
2020-08-04 2
2020-08-04 3
2020-08-04 3
2020-08-04 3
2020-08-04 3
2020-08-04 3
2020-08-04 3
2020-08-04 4
2020-08-04 4
2020-08-04 4
2020-08-04 4
2020-08-04 4
2020-08-04 4
2020-08-05 2
2020-08-05 2
2020-08-05 2
2020-08-05 2
2020-08-05 2
2020-08-05 3
2020-08-05 3
2020-08-05 3
2020-08-05 3
2020-08-05 3
2020-08-05 3
2020-08-05 4
2020-08-05 4
2020-08-05 4
2020-08-05 4
2020-08-05 4
2020-08-05 4
2020-08-06 2
2020-08-06 2
2020-08-06 2
2020-08-06 2
2020-08-06 2
2020-08-06 3
2020-08-06 3
2020-08-06 3
2020-08-06 3
2020-08-06 3
2020-08-06 3
2020-08-06 4
2020-08-06 4
2020-08-06 4
2020-08-06 4
2020-08-06 4
2020-08-06 4
2020-08-07 2
2020-08-07 2
2020-08-07 2
2020-08-07 2
2020-08-07 2
2020-08-07 3
2020-08-07 3
2020-08-07 3
2020-08-07 3
2020-08-07 3

2020-11-10 2
2020-11-10 3
2020-11-10 3
2020-11-10 3
2020-11-10 3
2020-11-10 3
2020-11-10 3
2020-11-10 4
2020-11-10 4
2020-11-10 4
2020-11-10 4
2020-11-10 4
2020-11-11 2
2020-11-11 2
2020-11-11 2
2020-11-11 2
2020-11-11 2
2020-11-11 3
2020-11-11 3
2020-11-11 3
2020-11-11 3
2020-11-11 3
2020-11-11 3
2020-11-11 4
2020-11-11 4
2020-11-11 4
2020-11-11 4
2020-11-11 4
2020-11-12 2
2020-11-12 2
2020-11-12 2
2020-11-12 2
2020-11-12 2
2020-11-12 3
2020-11-12 3
2020-11-12 3
2020-11-12 3
2020-11-12 3
2020-11-12 3
2020-11-12 4
2020-11-12 4
2020-11-12 4
2020-11-12 4
2020-11-12 4
2020-11-13 2
2020-11-13 2
2020-11-13 2
2020-11-13 2
2020-11-13 2
2020-11-13 3
2020-11-13 3
2020-11-13 3
2020-11-13 3
2020-11-13 3
2020-11-13 3
2020-11-13 4
2020-11-13 4
2020-11-13 4
2020-11-13 4
2020-11-13 4
2020-11-13 4
2020-11-16 2
2020-11-16 2
2020-11-16 2
2020-11-16 2
2020-11-16 2
2020-11-16 3
2020-11-16 3
2020-11-16 3
2020-11-16 3
2020-11-16 3
2020-11-16 3
2020-11-16 4
2020-11-16 4
2020-11-16 4
2020-11-16 4
2020-11-16 4

2021-02-22 2
2021-02-22 2
2021-02-22 2
2021-02-22 2
2021-02-22 3
2021-02-22 3
2021-02-22 3
2021-02-22 3
2021-02-22 3
2021-02-22 3
2021-02-22 4
2021-02-22 4
2021-02-22 4
2021-02-22 4
2021-02-22 4
2021-02-23 2
2021-02-23 2
2021-02-23 2
2021-02-23 2
2021-02-23 2
2021-02-23 3
2021-02-23 3
2021-02-23 3
2021-02-23 3
2021-02-23 3
2021-02-23 3
2021-02-23 4
2021-02-23 4
2021-02-23 4
2021-02-23 4
2021-02-23 4
2021-02-24 2
2021-02-24 2
2021-02-24 2
2021-02-24 2
2021-02-24 2
2021-02-24 3
2021-02-24 3
2021-02-24 3
2021-02-24 3
2021-02-24 3
2021-02-24 3
2021-02-24 4
2021-02-24 4
2021-02-24 4
2021-02-24 4
2021-02-24 4
2021-02-25 2
2021-02-25 2
2021-02-25 2
2021-02-25 2
2021-02-25 2
2021-02-25 3
2021-02-25 3
2021-02-25 3
2021-02-25 3
2021-02-25 3
2021-02-25 3
2021-02-25 4
2021-02-25 4
2021-02-25 4
2021-02-25 4
2021-02-25 4
2021-02-26 2
2021-02-26 2
2021-02-26 2
2021-02-26 2
2021-02-26 2
2021-02-26 3
2021-02-26 3
2021-02-26 3
2021-02-26 3
2021-02-26 3
2021-02-26 3
2021-02-26 4
2021-02-26 4
2021-02-26 4

In [10]:
pd.read_pickle('./data/processed/universe.pkl')

,itemcode,itemname,price,nav,volumne_sh,trading_amt_mln,market_cap_100m,exchange_nm,market,asset_class,instruments,exposure,dc_risky_asset
0,C000001,현금,1,1.0,NaN,NaN,NaN,NaN,Domestic,Cash,Cash,Cash,1.0
1,D000001,예금,1,1.0,NaN,NaN,NaN,NaN,Domestic,Cash,Deposit,Deposit,1.0
2,A069500,KODEX 200,42095,42105.0,5426223.0,228445.0,51545.00,KRX,Domestic,Equity,ETF,Large,0.7
3,A277630,TIGER 코스피,31060,31050.0,4776.0,148.0,637.00,KRX,Domestic,Equity,ETF,Broad,0.7
4,A292160,TIGER KRX300,19250,19270.0,60658.0,1163.0,327.00,KRX,Domestic,Equity,ETF,Large,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,A130730,KOSEF 단기자금,100970,100968.0,211753.0,21380.0,486.00,KRX,Domestic,Fixed Income,ETF,Money,1.0
95,A122260,KOSEF 통안채1년,101170,101173.0,193096.0,19536.0,475.00,KRX,Domestic,Fixed Income,ETF,ShortD,1.0
96,A329750,TIGER 미국달러단기채권액티브,9645,NaN,78822.0,760.0,378.00,KRX,USA,Alternative,ETF,Money,1.0
97,A196230,KBSTAR 단기통안채,104325,104325.0,388477.0,40527.0,1847.00,KRX,Domestic,Fixed Income,ETF,Money,1.0


In [12]:
['a','b'] + ['c']

['a', 'b', 'c']

In [ ]:
def get_advised_portfolios